In [1]:
from granite_io.backend.vllm_server import LocalVLLMServer
from granite_io.io.granite_3_3.input_processors.granite_3_3_input_processor import (
    Granite3Point3Inputs,
)

In [2]:
from granite_io.io.hyde_rewrite import HyDERewriteIOProcessor

In [3]:
model_name = "ibm-granite/granite-3.3-8b-instruct"

In [4]:
server = LocalVLLMServer(model_name)

INFO 01:51:21 Running: /dccstor/vineeku6-seq2seq/code/granite-io/.venv/bin/vllm serve ibm-granite/granite-3.3-8b-instruct --port 48007 --gpu-memory-utilization 0.45 --max-model-len 32768 --guided_decoding_backend outlines --device auto --enforce-eager


In [5]:
server.wait_for_startup(200)

INFO 06-07 01:52:10 [__init__.py:239] Automatically detected platform cuda.
INFO 06-07 01:52:23 [api_server.py:1043] vLLM API server version 0.8.5.post1
INFO 06-07 01:52:23 [api_server.py:1044] args: Namespace(subparser='serve', model_tag='ibm-granite/granite-3.3-8b-instruct', config='', host=None, port=48007, uvicorn_log_level='info', disable_uvicorn_access_log=False, allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, enable_ssl_refresh=False, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_request_id_headers=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='ibm-granite/granite-3.3-8b-instruct', task='auto', tokenizer=None, hf_conf

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:07<00:23,  7.93s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:15<00:15,  7.77s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:17<00:05,  5.27s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:25<00:00,  6.13s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:25<00:00,  6.34s/it]



INFO 06-07 01:53:30 [loader.py:458] Loading weights took 25.47 seconds
INFO 06-07 01:53:31 [model_runner.py:1140] Model loading took 15.2512 GiB and 26.748777 seconds
INFO 06-07 01:53:34 [worker.py:287] Memory profiling takes 3.16 seconds
INFO 06-07 01:53:34 [worker.py:287] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.45) = 35.66GiB
INFO 06-07 01:53:34 [worker.py:287] model weights take 15.25GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 3.35GiB; the rest of the memory reserved for KV Cache is 16.97GiB.
INFO 06-07 01:53:34 [executor_base.py:112] # cuda blocks: 6950, # CPU blocks: 1638
INFO 06-07 01:53:34 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 3.39x
INFO 06-07 01:53:38 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 6.95 seconds
INFO 06-07 01:53:38 [api_server.py:1090] Starting vLLM API server on http://0.0.0.0:48007
INFO 06-07 01:53:38 [launcher.py:28]

INFO:     Started server process [2968748]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     127.0.0.1:57620 - "GET /ping HTTP/1.1" 200 OK


In [6]:
backend = server.make_backend()

In [7]:
# Create an example chat completion with a short conversation.
chat_input = Granite3Point3Inputs.model_validate(
    {
        "messages": [
            {"role": "assistant", "content": "Welcome to pet questions!"},
            {
                "role": "user",
                "content": "I have two pets, a dog named Rex and a cat named Lucy.",
            },
            {
                "role": "assistant",
                "content": "Great, what would you like to share about them?",
            },
            {
                "role": "user",
                "content": "Rex spends a lot of time in the backyard and outdoors, "
                "and Luna is always inside.",
            },
            {
                "role": "assistant",
                "content": "Sounds good! Rex must love exploring outside, while Lucy "
                "probably enjoys her cozy indoor life.",
            },
            {
                "role": "user",
                "content": "But is he more likely to get fleas because of that?",
            },
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)
chat_input

Granite3Point3Inputs(messages=[AssistantMessage(content='Welcome to pet questions!', role='assistant', tool_calls=[], reasoning_content=None, citations=None, documents=None, hallucinations=None, stop_reason=None), UserMessage(content='I have two pets, a dog named Rex and a cat named Lucy.', role='user'), AssistantMessage(content='Great, what would you like to share about them?', role='assistant', tool_calls=[], reasoning_content=None, citations=None, documents=None, hallucinations=None, stop_reason=None), UserMessage(content='Rex spends a lot of time in the backyard and outdoors, and Luna is always inside.', role='user'), AssistantMessage(content='Sounds good! Rex must love exploring outside, while Lucy probably enjoys her cozy indoor life.', role='assistant', tool_calls=[], reasoning_content=None, citations=None, documents=None, hallucinations=None, stop_reason=None), UserMessage(content='But is he more likely to get fleas because of that?', role='user')], tools=[], generate_inputs=Ge

In [8]:
io_proc = HyDERewriteIOProcessor(backend)

In [9]:
chat_result = await io_proc.acreate_chat_completion(chat_input)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

INFO 06-07 01:53:42 [logger.py:39] Received request cmpl-edba0feabe88457eb6055fdf051ea0e9-0: prompt: "<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.\nToday's Date: June 07, 2025.\nYou are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>Generate a short answer for the user query below. Do not use more than 50 words.\nBut is he more likely to get fleas because of that?<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>", params: SamplingParams(n=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, stop=[], stop_token_ids=[], bad_words=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=256, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None, guided_decoding=None, extra_args=None), prompt_token_ids: [49152, 2946, 

In [10]:
chat_result

ChatCompletionResults(results=[ChatCompletionResult(next_message=UserMessage(content='But is he more likely to get fleas because of that? Yes, pets with longer fur or those frequently in contact with infested areas are more prone to flea infestations. Regular grooming and preventive treatments can help minimize this risk.', role='user'))])

INFO 06-07 01:53:53 [metrics.py:486] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 2.5 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.
INFO 06-07 01:54:03 [metrics.py:486] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.
